In [2]:
import os
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras import layers
import tensorflow.keras as keras
from PIL import Image
import numpy as np

In [3]:
src_dir = "../datasets/ori/"
train_dir = "../datasets/train/"
val_dir = "../datasets/validation/"
test_dir = "../datasets/test/"
categories = ['igneous/', 'metamorphic/', 'sedimentary/']
for category in categories:
    file_list = os.listdir(src_dir+category)
    for n, i in enumerate(file_list[:200]):
        img = Image.open(src_dir+category+i)
        img_resize = img.resize((150, 150))
        img_resize.save(train_dir+category[:-1]+"_"+str(n)+".jpg", "JPEG")
    for n, i in enumerate(file_list[200:300]):
        img = Image.open(src_dir+category+i)
        img_resize = img.resize((150, 150))
        img_resize.save(val_dir+category[:-1]+"_"+str(n)+".jpg", "JPEG")
    for n, i in enumerate(file_list[300:400]):
        img = Image.open(src_dir+category+i)
        img_resize = img.resize((150, 150))
        img_resize.save(test_dir+category[:-1]+"_"+str(n)+".jpg", "JPEG")

In [4]:
train_data = np.empty(shape=(1, 150, 150, 3), dtype=np.float64)
for i in os.listdir(train_dir):
    img = Image.open(train_dir+i)
    train_data = np.append(train_data, [np.asarray(img)], axis=0)
train_data = np.delete(train_data, [0,0], axis=0)
train_data = train_data / 255.0

In [5]:
a = [0 for i in range(200)] + [1 for j in range(200)] + [2 for k in range(200)]
train_labels = np.array(a)

In [6]:
s = np.arange(600)
np.random.shuffle(s)
train_data = train_data[s]
train_labels = train_labels[s]
print(train_data.shape, train_labels.shape)

(600, 150, 150, 3) (600,)


In [13]:
with tf.device('/CPU:0'):
    inputs = keras.Input(shape=(150, 150, 3,))
    flat = layers.Flatten()(inputs)
    fc1 = layers.Dense(64, activation='relu')(flat)
    fc2 = layers.Dense(128, activation='relu')(fc1)
    fc3 = layers.Dense(256, activation='relu')(fc2)
    output = layers.Dense(3, activation='softmax')(fc3)
    model = Model(inputs=inputs, outputs=output, name="FC_Model")
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])
model.summary()

Model: "FC_Model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 flatten_1 (Flatten)         (None, 67500)             0         
                                                                 
 dense_4 (Dense)             (None, 64)                4320064   
                                                                 
 dense_5 (Dense)             (None, 128)               8320      
                                                                 
 dense_6 (Dense)             (None, 256)               33024     
                                                                 
 dense_7 (Dense)             (None, 3)                 771       
                                                                 
Total params: 4,362,179
Trainable params: 4,362,179
Non-tr

In [14]:
with tf.device('/CPU:0'):
    model.fit(train_data, train_labels, epochs=100)

Epoch 1/100
19/19 [==============================] - 1s 56ms/step - loss: 4.9736 - acc: 0.3350
Epoch 2/100
19/19 [==============================] - 1s 65ms/step - loss: 1.5437 - acc: 0.3483
Epoch 3/100
19/19 [==============================] - 1s 65ms/step - loss: 1.7893 - acc: 0.3167
Epoch 4/100
19/19 [==============================] - 1s 59ms/step - loss: 1.3639 - acc: 0.3333
Epoch 5/100
19/19 [==============================] - 1s 50ms/step - loss: 1.2348 - acc: 0.3317
Epoch 6/100
19/19 [==============================] - 1s 45ms/step - loss: 1.1747 - acc: 0.3433
Epoch 7/100
19/19 [==============================] - 1s 57ms/step - loss: 1.1447 - acc: 0.3517
Epoch 8/100
19/19 [==============================] - 1s 55ms/step - loss: 1.1036 - acc: 0.4067
Epoch 9/100
19/19 [==============================] - 1s 48ms/step - loss: 1.1487 - acc: 0.3533
Epoch 10/100
19/19 [==============================] - 1s 53ms/step - loss: 1.1045 - acc: 0.3683
Epoch 11/100
19/19 [=============================

In [15]:
with tf.device('/CPU:0'):
    inputs = keras.Input(shape=(150, 150, 3,))
    conv1 = layers.Conv2D(64, (3, 3), activation='relu')(inputs)
    pool1 = layers.MaxPool2D((2, 2))(conv1)
    drop1 = layers.Dropout(0.4)(pool1)
    conv2 = layers.Conv2D(128, (3, 3), activation='relu')(drop1)
    pool2 = layers.MaxPool2D((2, 2))(conv2)
    drop2 = layers.Dropout(0.4)(pool2)
    conv3 = layers.Conv2D(256, (3, 3), activation='relu')(drop2)
    flat = layers.Flatten()(conv3)
    output = layers.Dense(3, activation='softmax')(flat)
    model = Model(inputs=inputs, outputs=output, name="CNN")
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])
model.summary()

Model: "CNN"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 148, 148, 64)      1792      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 74, 74, 64)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 74, 74, 64)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 72, 72, 128)       73856     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 36, 36, 128)      0         
 2D)                                                           

In [16]:
model.fit(train_data, train_labels, epochs=30, validation_split=0.2)

Epoch 1/30


2021-12-12 03:18:22.788233: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8300
2021-12-12 03:18:23.387379: E tensorflow/stream_executor/cuda/cuda_blas.cc:226] failed to create cublas handle: CUBLAS_STATUS_ALLOC_FAILED
2021-12-12 03:18:23.387797: W tensorflow/core/framework/op_kernel.cc:1745] OP_REQUIRES failed at conv_ops.cc:1120 : NOT_FOUND: No algorithm worked!


NotFoundError: 2 root error(s) found.
  (0) NOT_FOUND:  No algorithm worked!
	 [[node CNN/conv2d/Conv2D
 (defined at /usr/lib/python3.9/site-packages/keras/layers/convolutional.py:231)
]]
	 [[Adam/IdentityN/_60]]
  (1) NOT_FOUND:  No algorithm worked!
	 [[node CNN/conv2d/Conv2D
 (defined at /usr/lib/python3.9/site-packages/keras/layers/convolutional.py:231)
]]
0 successful operations.
0 derived errors ignored. [Op:__inference_train_function_10639]

Errors may have originated from an input operation.
Input Source operations connected to node CNN/conv2d/Conv2D:
In[0] IteratorGetNext (defined at /usr/lib/python3.9/site-packages/keras/engine/training.py:866)	
In[1] CNN/conv2d/Conv2D/ReadVariableOp:

Operation defined at: (most recent call last)
>>>   File "/usr/lib/python3.9/runpy.py", line 197, in _run_module_as_main
>>>     return _run_code(code, main_globals, None,
>>> 
>>>   File "/usr/lib/python3.9/runpy.py", line 87, in _run_code
>>>     exec(code, run_globals)
>>> 
>>>   File "/usr/

In [7]:
base_model = keras.applications.VGG16(input_shape=(150, 150, 3),
                                      include_top=False,
                                      weights='imagenet')
base_model.trainable = False
base_model.summary()

2021-12-16 10:31:49.945224: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-16 10:31:50.009547: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-16 10:31:50.009891: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-16 10:31:50.010988: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler fl

58900480/58889256 [==============================] - 1s 0us/step
Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 150, 150, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 150, 150, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 75, 75, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 75, 75, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 75, 75, 128)       147584    
                                                              

In [ ]:
top_model = keras.models.Sequential()
top_model.add(layers.Flatten(input_shape=base_model.output_shape[1:]))
top_model.add(layers.Dense(256, activation='relu'))
top_model.add(layers.Dense(512, activation='relu'))
top_model.add(layers.Dense(3, activation='softmax'))